In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

c:\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [91]:
df_fhvhv = spark.read.parquet('fhvhv/*/*')

In [92]:
df_fhvhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [93]:
df_fhvhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [94]:
df_fhvhv.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [95]:
df_fhvhv.registerTempTable('trips_fhvhv')

C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [97]:
#Question 3
spark.sql("""
SELECT
    count(*)
FROM
    trips_fhvhv
WHERE
    month(pickup_datetime)=6 AND day(pickup_datetime)=15
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [121]:
#Question 4
spark.sql("""
SELECT
    pickup_datetime,
    dropoff_datetime,
    dropoff_datetime-pickup_datetime as Difference,
    DATEDIFF(hour, pickup_datetime, dropoff_datetime) AS InHours
FROM
    trips_fhvhv
ORDER BY
    Difference DESC
""").show()

+-------------------+-------------------+--------------------+-------+
|    pickup_datetime|   dropoff_datetime|          Difference|InHours|
+-------------------+-------------------+--------------------+-------+
|2021-06-25 13:55:41|2021-06-28 08:48:25|INTERVAL '2 18:52...|     66|
|2021-06-22 12:09:45|2021-06-23 13:42:44|INTERVAL '1 01:32...|     25|
|2021-06-27 10:32:29|2021-06-28 06:31:20|INTERVAL '0 19:58...|     19|
|2021-06-26 22:37:11|2021-06-27 16:49:01|INTERVAL '0 18:11...|     18|
|2021-06-23 20:40:43|2021-06-24 13:08:44|INTERVAL '0 16:28...|     16|
|2021-06-23 22:03:31|2021-06-24 12:19:39|INTERVAL '0 14:16...|     14|
|2021-06-24 23:11:00|2021-06-25 13:05:35|INTERVAL '0 13:54...|     13|
|2021-06-04 20:56:02|2021-06-05 08:36:14|INTERVAL '0 11:40...|     11|
|2021-06-27 07:45:19|2021-06-27 19:07:16|INTERVAL '0 11:21...|     11|
|2021-06-20 17:05:12|2021-06-21 04:04:16|INTERVAL '0 10:59...|     10|
|2021-06-01 12:25:29|2021-06-01 22:41:32|INTERVAL '0 10:16...|     10|
|2021-

In [98]:
spark.sql("""
SELECT
    PULocationID,
    count(1)
FROM
    trips_fhvhv
GROUP BY 
    PULocationID
ORDER BY
    count(1) DESC
""").show()

+------------+--------+
|PULocationID|count(1)|
+------------+--------+
|          61|  231279|
|          79|  221244|
|         132|  188867|
|          37|  187929|
|          76|  186780|
|         231|  164344|
|         138|  161596|
|         234|  158937|
|         249|  154698|
|           7|  152493|
|         148|  151020|
|          68|  147673|
|          42|  146402|
|         255|  143683|
|         181|  143594|
|         225|  141427|
|          48|  139611|
|         246|  139431|
|          17|  138428|
|         170|  137879|
+------------+--------+
only showing top 20 rows



In [99]:
df_zones = spark.read.csv('taxi_zone_lookup.csv', header=True)

In [100]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [101]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [102]:
df_zones.registerTempTable('zones')

In [103]:
spark.sql("""
SELECT
    LocationID,
    Borough,
    Zone,
    service_zone
FROM
    zones
""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [109]:
#Question 6
spark.sql("""
SELECT
    PULocationID,
    count(1)
FROM
    trips_fhvhv
GROUP BY 
    PULocationID
ORDER BY
    count(1) DESC
LIMIT 1
""").show()

+------------+--------+
|PULocationID|count(1)|
+------------+--------+
|          61|  231279|
+------------+--------+



In [106]:
spark.sql("""
SELECT
    trips_fhvhv.PULocationID, zones.Borough, zones.Zone, zones.service_zone
FROM
    trips_fhvhv
FULL JOIN
    zones
ON
    trips_fhvhv.PULocationID = zones.LocationID
WHERE
    zones.LocationID = 61
LIMIT 1
""").show()

+------------+--------+-------------------+------------+
|PULocationID| Borough|               Zone|service_zone|
+------------+--------+-------------------+------------+
|          61|Brooklyn|Crown Heights North|   Boro Zone|
+------------+--------+-------------------+------------+

